# Notebook 5: Agentic RAG with Tool Orchestration

This notebook demonstrates:
1. Defining agent tools (Retriever, Citation Checker, Summarizer)
2. Orchestrating multi-tool workflows
3. LangGraph agent implementation
4. Execution tracing
5. Tool usage patterns

In [ ]:
# Import required modules
import os
from dotenv import load_dotenv
from src.agent_rag import (
    AgentRunner,
    RetrieverTool,
    CitationCheckerTool,
    SummarizerTool,
    create_langgraph_agent
)

load_dotenv()
print("✓ Imports successful")

## Step 1: Initialize Individual Tools

In [ ]:
# Create individual tools
retriever = RetrieverTool(
    faiss_path="faiss_index/",
    openai_api_key=os.getenv("OPENAI_API_KEY"),
    top_k=3
)

citation_checker = CitationCheckerTool()

summarizer = SummarizerTool(
    openai_api_key=os.getenv("OPENAI_API_KEY")
)

print("✓ Tools initialized:")
print(f"  - {retriever.name}: {retriever.description}")
print(f"  - {citation_checker.name}: {citation_checker.description}")
print(f"  - {summarizer.name}: {summarizer.description}")

## Step 2: Test Individual Tools

In [ ]:
# Test Retriever
query = "What are data subject rights?"
docs = retriever.run(query)

print(f"Retriever Results for '{query}':")
for i, doc in enumerate(docs, 1):
    print(f"  {i}. Article {doc['metadata']['article']}: Score {doc['score']:.2f}")
    print(f"     {doc['content'][:80]}...")

In [ ]:
# Test Citation Checker
claim = "GDPR Article 15 grants the right to access personal data"
sources = docs

verification = citation_checker.run({
    "claim": claim,
    "sources": sources
})

print(f"\nCitation Verification:")
print(f"  Verified: {verification['verified']}")
print(f"  Confidence: {verification['confidence']:.2%}")
print(f"  Supporting sources: {verification['num_supporting']}/{verification['num_total']}")

In [ ]:
# Test Summarizer
summary = summarizer.run({
    "documents": docs,
    "focus": "data subject rights"
})

print(f"\nSummary:")
print(summary)

## Step 3: Create Agent Runner

In [ ]:
# Initialize agent with all tools
agent = AgentRunner(
    faiss_path="faiss_index/",
    openai_api_key=os.getenv("OPENAI_API_KEY")
)

print(f"✓ Agent runner initialized with {len(agent.tools)} tools")
for tool in agent.tools:
    print(f"  - {tool.name}")

## Step 4: Run Complete Agentic Workflow

In [ ]:
# Run agent on a query
query = "What are the main data subject rights under GDPR?"

result = agent.run(query)

print(f"\nQuery: {result['query']}")
print(f"\n{'='*60}")
print("Answer:")
print(result['answer'])
print(f"{'='*60}")

## Step 5: Analyze Execution Trace

In [ ]:
# Display execution trace
print("\nExecution Trace:")
print(f"{'='*60}")

for i, step in enumerate(result['trace'], 1):
    print(f"\nStep {i}: {step['step'].upper()}")
    print(f"  Tool: {step['tool']}")
    print(f"  Result: {step['result']}")

print(f"\n{'='*60}")

## Step 6: View Detailed Results

In [ ]:
# Display detailed verification results
print("\nVerification Details:")
verification = result['verification']
print(f"  Verified: {verification['verified']}")
print(f"  Confidence: {verification['confidence']:.2%}")
print(f"  Supporting sources: {verification['num_supporting']}/{verification['num_total']}")

# Display summary
print(f"\n{'='*60}")
print("Summary:")
print(result['summary'])
print(f"{'='*60}")

## Step 7: Test Multiple Queries

In [ ]:
# Test with different query types
test_queries = [
    "What is the right to erasure?",
    "Explain data portability rights",
    "What are the lawful bases for processing?"
]

for query in test_queries:
    print(f"\n{'='*60}")
    print(f"Query: {query}")
    print(f"{'='*60}")
    
    result = agent.run(query)
    
    # Show summary info
    print(f"\nDocuments retrieved: {len(result['documents'])}")
    print(f"Verification confidence: {result['verification']['confidence']:.2%}")
    print(f"Trace steps: {len(result['trace'])}")

## Step 8: Create LangGraph Agent (Advanced)

In [ ]:
# Create LangGraph agent structure
tools = [retriever, citation_checker, summarizer]

langgraph_agent = create_langgraph_agent(
    tools=tools,
    openai_api_key=os.getenv("OPENAI_API_KEY")
)

print("\nLangGraph Agent:")
print(f"  Status: {langgraph_agent['status']}")
print(f"  Tools: {[tool.name for tool in langgraph_agent['tools']]}")
print("\nNote: In production, this would be a compiled LangGraph StateGraph")

## Summary

In this notebook, we:
- ✓ Created specialized agent tools
- ✓ Tested each tool independently
- ✓ Orchestrated multi-tool workflows
- ✓ Analyzed execution traces
- ✓ Introduced LangGraph agent structure

Next: Notebook 6 - Graph-Enhanced RAG